In [1]:
# get parquet data

!wget https://github.com/IBM/coursera/raw/master/hmp.parquet
    
df = spark.read.parquet('hmp.parquet')

df.show(10)

Waiting for a Spark session to start...
Spark Initialization Done! ApplicationId = app-20200213033745-0000
KERNEL_ID = 1c8189d2-a097-48ba-8018-f588d0f12eae
--2020-02-13 03:37:48--  https://github.com/IBM/coursera/raw/master/hmp.parquet
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet [following]
--2020-02-13 03:37:48--  https://raw.githubusercontent.com/IBM/coursera/master/hmp.parquet
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 932997 (911K) [application/octet-stream]
Saving to: 'hmp.parquet'

100%[======================================>] 932,997     --.-K/s   in 0.04s   

2020-02-13 03:37:49 (25.

In [2]:
df.createOrReplaceTempView('df')

In [3]:
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler, MinMaxScaler
from pyspark.ml import Pipeline

indexer = StringIndexer(inputCol='class', outputCol='class_index')
encoder = OneHotEncoder(inputCol='class_index', outputCol='class_encoded')

vectorAssembler = VectorAssembler(inputCols=['x','y','z'], outputCol='features')
rescaler = MinMaxScaler(inputCol='features', outputCol='feature_rescale')

# create pipeline
preprocessing = Pipeline(stages=[indexer, encoder, vectorAssembler, rescaler])
processed_df = preprocessing.fit(df).transform(df)

processed_df.show(10)

+---+---+---+--------------------+-----------+-----------+--------------+----------------+--------------------+
|  x|  y|  z|              source|      class|class_index| class_encoded|        features|     feature_rescale|
+---+---+---+--------------------+-----------+-----------+--------------+----------------+--------------------+
| 22| 49| 35|Accelerometer-201...|Brush_teeth|        6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 49| 35|Accelerometer-201...|Brush_teeth|        6.0|(13,[6],[1.0])|[22.0,49.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|        6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 22| 52| 35|Accelerometer-201...|Brush_teeth|        6.0|(13,[6],[1.0])|[22.0,52.0,35.0]|[0.34920634920634...|
| 21| 52| 34|Accelerometer-201...|Brush_teeth|        6.0|(13,[6],[1.0])|[21.0,52.0,34.0]|[0.33333333333333...|
| 22| 51| 34|Accelerometer-201...|Brush_teeth|        6.0|(13,[6],[1.0])|[22.0,51.0,34.0]|[0.34920634920

In [ ]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator

kmeans = KMeans(featureCol='feature_rescale').setK(14).setSeed(2020)
pipeline = Pipeline(stages=[preprocessing, kmeans])
model = pipeline.fit(df)

